# Regresión: 1 neurona con varias columnas en X


## 1. Dataset con X, y

Supongamos un problema de **regresión** muy sencillo:

- **Entrada**: Años de experiencia y posición ($ x $)  
- **Salida**: Salario ($ y $)

La idea es muy similar al caso de una sola entrada, pero ahora tenemos un **vector de pesos** en lugar de un solo escalar.

In [1]:
import numpy as np

# Ejemplo de X con 2 columnas:
#  - Primera columna: años de experiencia
#  - Segunda columna: nivel de estudios (codificado numéricamente)
X = np.array([
    [1, 0],   # persona1: 1 año exp, nivel 0
    [2, 0],   # persona2: 2 años exp, nivel 0
    [3, 1],   # persona3: 3 años exp, nivel 1
    [5, 1],   # persona4: 5 años exp, nivel 1
    [6, 2],   # persona5: 6 años exp, nivel 2
], dtype=float)

y = np.array([20, 25, 35, 45, 60], dtype=float)  # salarios correspondientes

## 2. Definir una neurona

Imagina que para cada ejemplo $i$:

- $ x_1^{(i)} $ = Años de experiencia  
- $ x_2^{(i)} $ = Nivel de estudios (codificado como algún número entero, 0, 1, 2, etc.)

El **modelo lineal** (la neurona) dice:

$$
\hat{y}^{(i)} 
= w_1 \cdot x_1^{(i)} + w_2 \cdot x_2^{(i)} + b,
$$

donde:
- $ w_1 $ y $ w_2 $ son los **pesos**,
- $ b $ es el **sesgo** (bias),
- $\hat{y}^{(i)}$ es la **predicción** del salario para el ejemplo $i$.

En forma vectorizada, si $\mathbf{x}^{(i)} = (x_1^{(i)}, x_2^{(i)})$ y $\mathbf{w} = (w_1, w_2)$, se escribe:

$$
\hat{y}^{(i)} = \mathbf{w}^\top \mathbf{x}^{(i)} + b.
$$

Calcula $ \hat{y} = \mathbf{w}^\top \mathbf{x} + b $ para **todos** los ejemplos a la vez:

In [2]:
def forward(X, w, b):
    """
    X: shape (N, 2)  (N filas, 2 columnas)
    w: shape (2,)    
    b: escalar
    Return: y_pred de shape (N,)
    """
    return np.dot(X, w) + b  # w[0]*X[:,0] + w[1]*X[:,1] + b

## 3. Función de costo (MSE)

Si tenemos $N$ ejemplos en total, definimos la **pérdida MSE** como:

$$
\text{MSE}(\mathbf{w}, b) 
= \frac{1}{N} \sum_{i=1}^{N} \bigl(\hat{y}^{(i)} - y^{(i)}\bigr)^{2},
$$
donde $y^{(i)}$ es el salario real en el ejemplo $i$.


In [3]:
def mse_loss(y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

## 4. Derivadas de w y b

Para actualizar $\mathbf{w}$ y $b$ con **descenso por gradiente**, necesitamos las **derivadas** de la MSE respecto a cada parámetro.

1. **Gradiente respecto a $\mathbf{w}$** (en forma vectorial):
   $$
   \nabla_{\mathbf{w}} \text{MSE} 
   = \frac{2}{N} \sum_{i=1}^N 
     \bigl(\hat{y}^{(i)} - y^{(i)}\bigr) \, \mathbf{x}^{(i)}.
   $$
   - Observa que $\hat{y}^{(i)} = \mathbf{w}^\top \mathbf{x}^{(i)} + b$.

2. **Gradiente respecto a $b$**:
   $$
   \frac{\partial \text{MSE}}{\partial b}
   = \frac{2}{N} \sum_{i=1}^N 
     \bigl(\hat{y}^{(i)} - y^{(i)}\bigr).
   $$

En forma **vectorizada**:

$$
\nabla_{\mathbf{w}} \text{MSE}(\mathbf{w}, b)
= \frac{2}{N} \, X^\top \bigl(\hat{y} - y\bigr).
$$
$$
\frac{\partial \text{MSE}}{\partial b}
= \frac{2}{N} \sum_{i=1}^N (\hat{y}^{(i)} - y^{(i)}).
$$

In [4]:
def grad_w(X, y, w, b):
    """
    Retorna la derivada (gradiente) respecto a w, que será un vector (2,).
    """
    N = len(X)
    y_pred = forward(X, w, b)            # shape (N,)
    # (y_pred - y) -> shape (N,)
    # np.dot(X.T, (y_pred - y)) -> shape (2,) 
    # factor 2.0/N por la derivada del cuadrado y la media
    return (2.0 / N) * np.dot(X.T, (y_pred - y))

def grad_b(X, y, w, b):
    """
    Retorna la derivada respecto a b (un escalar).
    """
    N = len(X)
    y_pred = forward(X, w, b)
    return (2.0 / N) * np.sum(y_pred - y)

## Entrenamiento

Elegimos una **tasa de aprendizaje** (`learning_rate`), un número de **épocas** y repetimos:

1. Computar `y_pred`  
2. Calcular pérdida (MSE)  
3. Calcular `dw` y `db`  
4. Actualizar `w` y `b`

In [5]:
np.random.seed(42)
w = np.random.randn(2) * 0.01  # vector de 2 pesos pequeños
b = 0.0                        # iniciamos el sesgo en 0 (o un valor pequeño aleatorio)

learning_rate = 0.01
num_epochs = 500

for epoch in range(num_epochs):
    # 1. Forward
    y_pred = forward(X, w, b)
    
    # 2. Calcular costo (MSE)
    loss_value = mse_loss(y_pred, y)
    
    # 3. Gradientes
    dw = grad_w(X, y, w, b)  # vector shape (2,)
    db = grad_b(X, y, w, b)  # escalar
    
    # 4. Actualizar parámetros
    w -= learning_rate * dw
    b -= learning_rate * db
    
    # (Opcional) mostramos cada cierto número de épocas
    if (epoch+1) % 100 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss_value:.4f}, w: {w}, b: {b:.4f}")

# Resultados finales
print("\nEntrenamiento terminado.")
print(f"w final = {w}, b final = {b}")

Epoch 100/500 - Loss: 13.5768, w: [8.46925079 1.53604604], b: 5.6097
Epoch 200/500 - Loss: 8.4245, w: [8.00461021 1.4095659 ], b: 7.7951
Epoch 300/500 - Loss: 6.1775, w: [7.6164443  1.65838487], b: 9.2054
Epoch 400/500 - Loss: 4.9513, w: [7.28474243 2.07916559], b: 10.1665
Epoch 500/500 - Loss: 4.1463, w: [6.99693182 2.56376392], b: 10.8604

Entrenamiento terminado.
w final = [6.99693182 2.56376392], b final = 10.860388678533837


In [6]:
# Predicción final

y_pred_final = forward(X, w, b)
print("Predicciones finales:", y_pred_final)
print("Valores reales      :", y)


Predicciones finales: [17.8573205  24.85425232 34.41494806 48.4088117  57.96950744]
Valores reales      : [20. 25. 35. 45. 60.]


In [7]:
def mae_loss(y_pred, y_true):
    return np.mean(np.abs(y_pred - y_true))

mae_loss(y_pred_final, y)

np.float64(1.662556675631975)

In [8]:
def r2_score(y_pred, y_true):
    # Suma total de cuadrados (variación de los datos respecto a la media)
    total_variance = np.sum((y_true - np.mean(y_true)) ** 2)
    # Suma de los residuos al cuadrado (errores del modelo)
    residual_variance = np.sum((y_true - y_pred) ** 2)
    # R^2: 1 - (varianza explicada por el error / varianza total)
    return 1 - (residual_variance / total_variance)

r2_score(y_pred_final, y)

np.float64(0.9799053389864572)

In [9]:
X_nuevo = np.array([
    [4, 1.5]
], dtype=float)  # 1 sola fila, 2 columnas

y_nuevo_pred = forward(X_nuevo, w, b)
print("Predicción para X=[4, 1.5]:", y_nuevo_pred)

Predicción para X=[4, 1.5]: [42.69376184]
